En el corazón de todas las redes neuronales en PyTorch se encuentra el paquete **autograd**. Comencemos con una breve introducción.

El paquete **autograd** proporciona diferenciación automática para todas las **operaciones en tensores**. Es un marco definido en tiempo de ejecución, lo que significa que la retropropagación se basa en su código para determinar cómo comportarse, y cada iteración puede ser diferente.

Su incidencia en los tensores:

**torch.Tensor** es la clase central de este paquete. Si la condicion **.requires_grad** se establece en **True**, se rastrearán todas las operaciones en este tensor. Cuando se completa el cálculo, todos los gradientes se calculan automáticamente llamando a **.backward()** y todos los gradientes de este tensor se acumularán automáticamente en la propiedad **.grad.**

Para evitar que un tensor realice un seguimiento del historial, puede llamar al método **.detach()** para separarlo del historial de cálculo y deshabilitar el seguimiento de sus cálculos futuros.

De igual forma el seguimiento del historial (y el uso de la memoria) de un bloque de código se puede envolver con **torch.no_grad():**. Lo anterior es especialmente útil al evaluar modelos, ya que estos pueden tener parámetros entrenables con **require_grad=True**, pero no necesitamos cálculos de gradiente.

**Tensor** y **Function** están interconectados y construyen un gráfico acíclico que codifica una historia completa de computación. Cada tensor tiene un atributo **.grad_fn** que hace referencia a una función que ha creado **Tensor** (excepto los tensores creados por el usuario: cuando su **grad_fn** es **None**).

Si necesita calcular la derivada, puede llamar a **.backward()** en **Tensor**. Si **Tensor** es un escalar (es decir, contiene datos de un elemento), no necesita especificar ningún argumento **backward()** pero si tiene más elementos, debe especificar un argumento de gradiente (**gradient**) para que coincida con la forma del tensor.

**Para tener en cuenta**:
en otros artículos, puede ver que **Tensor** está envuelto en **Variable** para proporcionar un cálculo de gradiente automático. El llamado de **Variable** se marcó como caducado en la versión 0.41. Ahora puede usar **Tensor** directamente.

In [1]:
import torch

Creando un tensor y configurandolo como **require_grad=True** para rastrear su historial de cálculo

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


- Operar con tensores

In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


El resultado **y** se ha calculado, por lo que **grad_fn** se ha generado automáticamente

In [4]:
print(y.grad_fn)

Ahora realizar una operacion en **y**

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


*.requires_grad()*

**.requires_grad_(...)** puede cambiar la propiedad **require_grad** de un tensor existente. Si no se especifica, el indicador de entrada predeterminado es **False**.

In [6]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


# Gradientes

Backpropagation **out** debido a que es un escalar ==> **out.backward()** es igual a **out.backward(torch.tensor(1))**

In [7]:
out.backward()

In [8]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [9]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)


tensor([-194.4764, 1357.0542, 1086.9559], grad_fn=<MulBackward0>)


En la celda anterior **y** ya no es un escalar. **torch.autograd** no puede calcular directamente el jacobiano completo(vector-jacobian product), pero si solo queremos el producto vector-jacobiano, simplemente pasamos el vector como parámetro hacia atrás **backward:**

In [10]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


Si **.requires_grad=True** pero no desea realizar cálculos de **autograd** puede envolver la variable con **torch.no_grad():**

In [11]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


Para profundizar conceptos de **autograd**: https://pytorch.org/docs/autograd
    